# SPR 2026 - TF-IDF + TabPFN v2.5 (OTIMIZADO)

**TabPFN v2.5:** Transformer pré-treinado para classificação tabular - funciona sem treinamento!

**Otimizações aplicadas:**
1. ✅ TruncatedSVD: 5k features → 100 features densas
2. ✅ GPU acceleration: `device='cuda'`
3. ✅ Ensemble com subsampling (se dataset > 10k)

**Limitações do TabPFN:**
- Máximo 10.000 amostras de treino
- Máximo 500 features (usamos 100 para eficiência)
- Máximo 10 classes ✅ (temos 7)

**Tempo esperado:** ~5-10 min (inclui download do modelo)

---
**CONFIGURAÇÃO KAGGLE (OBRIGATÓRIO):**

1. **PRIMEIRO:** Acesse https://huggingface.co/Prior-Labs/tabpfn_2_5 e **aceite os termos de uso**
2. Settings → Internet → **ON**
3. Settings → Accelerator → **GPU T4 x2** (recomendado)
4. **Add Secret:**
   - Label: `HF_TOKEN`
   - Value: `<seu_token_huggingface_com_permissao_read>`
5. Execute "Run All" após commit
---

In [ ]:
# =============================================================================
# SPR 2026 - TFIDF + TABPFN v2.5 COM SVD (CÓDIGO CONSOLIDADO)
# =============================================================================
# IMPORTANTE: Aceite os termos em https://huggingface.co/Prior-Labs/tabpfn_2_5
# antes de executar este notebook!
# =============================================================================

import subprocess
import sys
import os

print("="*60)
print("SPR 2026 - TF-IDF + TabPFN v2.5 COM SVD")
print("="*60)

# ==== CONFIGURAR HUGGINGFACE TOKEN ====
print("\n[0/5] Configurando HuggingFace e instalando TabPFN v2.5...")

# Tentar pegar token do Kaggle Secrets
hf_token = None
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    hf_token = user_secrets.get_secret("HF_TOKEN")
    print("    ✓ Token HuggingFace carregado do Kaggle Secrets!")
except Exception as e:
    print(f"    ⚠️ Kaggle Secrets: {e}")

# Instalar dependências
subprocess.run([sys.executable, "-m", "pip", "install", "tabpfn", "huggingface_hub", "-q"], check=True)
print("    ✓ Pacotes instalados!")

# Login no HuggingFace (necessário para modelo gated)
from huggingface_hub import login
if hf_token:
    login(token=hf_token, add_to_git_credential=False)
    print("    ✓ Login HuggingFace realizado!")
else:
    print("    ⚠️ Token não encontrado. Configure 'HF_TOKEN' nos Kaggle Secrets")
    print("    ⚠️ E aceite os termos em: https://huggingface.co/Prior-Labs/tabpfn_2_5")

# ==== IMPORTS ====
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import warnings
warnings.filterwarnings('ignore')

from tabpfn import TabPFNClassifier
print("    ✓ TabPFN carregado!")

# ==== CONFIGURAÇÕES ====
SEED = 42
SVD_COMPONENTS = 100  # TabPFN funciona melhor com menos features
MAX_TRAIN_SIZE = 10000  # Limite do TabPFN
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
USE_GPU = torch.cuda.is_available()
np.random.seed(SEED)

print(f"    ✓ GPU disponível: {USE_GPU}")

# ==== CARREGAR DADOS ====
print("\n[1/5] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

# ==== TF-IDF ====
print("\n[2/5] Aplicando TF-IDF...")
tfidf = TfidfVectorizer(
    max_features=5000,  # Menor para SVD mais eficiente
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train_tfidf = tfidf.fit_transform(train['report'])
X_test_tfidf = tfidf.transform(test['report'])
y_train = train['target'].values
print(f"    TF-IDF esparso: {X_train_tfidf.shape}")

# ==== SVD - CRÍTICO PARA TABPFN ====
print(f"\n[3/5] Aplicando SVD: {X_train_tfidf.shape[1]} → {SVD_COMPONENTS} features...")
svd = TruncatedSVD(n_components=SVD_COMPONENTS, random_state=SEED)
X_train_svd = svd.fit_transform(X_train_tfidf)
X_test_svd = svd.transform(X_test_tfidf)
print(f"    Variância explicada: {svd.explained_variance_ratio_.sum():.2%}")
print(f"    ✅ Shape denso: {X_train_svd.shape} ← CONFIRME QUE TEM 100 COLUNAS!")

# Normalizar (importante para TabPFN)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_svd)
X_test = scaler.transform(X_test_svd)

# ==== TABPFN v2.5 ====
print(f"\n[4/5] Executando TabPFN v2.5 ({'GPU' if USE_GPU else 'CPU'})...")
device = 'cuda' if USE_GPU else 'cpu'

if len(X_train) <= MAX_TRAIN_SIZE:
    # Usar todos os dados diretamente
    print(f"    Dataset pequeno ({len(X_train)} amostras), usando todos...")
    
    model = TabPFNClassifier(device=device)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
else:
    # Ensemble com subsampling estratificado (dataset > 10k)
    print(f"    Dataset grande ({len(X_train)} amostras), usando ensemble com subsampling...")
    
    n_ensembles = 5
    all_preds = []
    
    splitter = StratifiedShuffleSplit(
        n_splits=n_ensembles, 
        train_size=MAX_TRAIN_SIZE, 
        random_state=SEED
    )
    
    for i, (train_idx, _) in enumerate(splitter.split(X_train, y_train)):
        print(f"    Ensemble {i+1}/{n_ensembles}...")
        
        X_subset = X_train[train_idx]
        y_subset = y_train[train_idx]
        
        model = TabPFNClassifier(device=device)
        model.fit(X_subset, y_subset)
        preds = model.predict_proba(X_test)
        all_preds.append(preds)
    
    # Média das probabilidades
    avg_probs = np.mean(all_preds, axis=0)
    predictions = np.argmax(avg_probs, axis=1)

print("    ✓ TabPFN executado!")

# ==== SUBMISSÃO ====
print("\n[5/5] Gerando submissão...")
submission = pd.DataFrame({
    'ID': test['ID'],
    'target': predictions
})
submission.to_csv('submission.csv', index=False)

print("="*60)
print("✅ CONCLUÍDO - submission.csv criado!")
print("="*60)
print("\nDistribuição das predições:")
print(submission['target'].value_counts().sort_index())